In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt

In [ ]:
cd = pd.read_excel('CustomersData.xlsx')
dc = pd.read_csv('Discount_Coupon.csv')
ms = pd.read_csv('Marketing_Spend.csv')
os = pd.read_csv('Online_Sales.csv')
ta = pd.read_excel('Tax_amount.xlsx')

os = pd.merge(left=os,right=cd,on='CustomerID', how='left')

os['Transaction_Date']=pd.to_datetime(os['Transaction_Date'], format='%Y%m%d')

os['month']=os['Transaction_Date'].dt.strftime("%b")

os.columns

dc.columns

os = pd.merge(left=os,right=dc,left_on=['month','Product_Category'],right_on=['Month','Product_Category '], how='left')

os = pd.merge(left=os,right=ta,on='Product_Category',how='left')

data=os.copy()

data.to_csv('data.csv',index=False)

In [ ]:
data=pd.read_csv('data.csv')

data.head()

In [ ]:
data['Transaction_ID'].nunique()

In [ ]:
data['Invoice_Value'] = round((data['Quantity']*data['Avg_Price']*(100-data['Discount_pct'])*(1+data['GST'])/100)+data['Delivery_Charges'],0)

In [ ]:
ms['Date']=pd.to_datetime(ms['Date'], format="%m/%d/%Y")

In [ ]:
data['Transaction_Date']=data['Transaction_Date'].astype('datetime64[ns]')

In [ ]:
datewise = pd.DataFrame(data.groupby('Transaction_Date')['Invoice_Value'].sum())
datewise.reset_index(inplace=True)

In [ ]:
datewise=pd.merge(left=datewise,left_on=['Transaction_Date'],right=ms, right_on =['Date'],how='left' )
datewise.drop(columns='Date',inplace=True)

In [ ]:
data.info()

In [ ]:
data.drop(columns=['Month','Product_Category '],inplace=True)

In [ ]:
col = data.columns.to_list()

In [ ]:
col.remove('Transaction_Date')
col.remove('Discount_pct')
col.remove('GST')
col.remove('Invoice_Value')
col.remove('Quantity')
col.remove('Avg_Price')
col.remove('Delivery_Charges')

In [ ]:
col

Customers Acquired Every Month

In [ ]:
for i in col:
    print(i,data[i].nunique())

In [ ]:
data['Coupon_Status'].unique()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.columns

In [ ]:
data['visitcount']=data.groupby('CustomerID')['Transaction_Date'].rank(method='dense')

In [ ]:
new_user = data.loc[:,['visitcount','month']][data['visitcount']==1].groupby('month')['visitcount'].count()

In [ ]:
new_user=pd.DataFrame(new_user)

In [ ]:
new_user.reset_index(inplace=True)

In [ ]:
ax = sns.barplot(x='month',y='visitcount', data =new_user)
ax.set(xlabel='Month', ylabel='No. of new users')

In [ ]:
start_date = data.groupby('CustomerID')['Transaction_Date'].min()

In [ ]:
start_date = start_date.rename("start_day")

In [ ]:
data = pd.merge(data, start_date, left_on='CustomerID', right_index=True)

In [ ]:
data['days_since_start'] = (data['Transaction_Date'] - data['start_day']).dt.days

In [ ]:
data['start_week'] = data.apply(lambda x: (x['start_day'] - pd.Timedelta(x['start_day'].dayofweek, u='Transaction_Date')).date(), axis=1)
data['start_week'] = pd.to_datetime(data['start_week'])
data['weeks_since_start'] = np.floor((data['Transaction_Date'] - data['start_week']) / np.timedelta64(1, 'W'))

In [ ]:
data['start_month'] = data['start_day'].values.astype('datetime64[M]')
data['months_since_start'] = np.floor((data['Transaction_Date'] - data['start_month']) / np.timedelta64(1, 'M'))

In [ ]:
data['days_since_start'].nunique()

In [ ]:
data.groupby('days_since_start')['CustomerID'].nunique().sum()

In [ ]:
data.groupby('days_since_start')['CustomerID'].nunique()

In [ ]:
whole_retention = data.groupby('days_since_start')['CustomerID'].nunique() / data['CustomerID'].nunique()

plt.plot(whole_retention[2:]*100)
plt.title("365-Day Daily Retention")
plt.xlabel("Days Since User First Started")
plt.ylabel("Percent Of Users Retained")
plt.ylim(0,1.05)


In [ ]:
plt.figure(figsize=(12,6))
legend_x = 85
legend_y = .55


for i, month in enumerate(data['start_month'].unique()):
    
    dfc = data[data['start_month']==pd.to_datetime(month)]

    retention_cohort = dfc.groupby('days_since_start')['CustomerID'].nunique() / dfc['CustomerID'].nunique()
    
    p = plt.plot(retention_cohort[1:]*20)
    
    plt.text(s=pd.to_datetime(month).strftime('%B %Y'), c=p[0].get_color(),
             x=legend_x, y=legend_y + (.05 * i), weight='bold')

plt.text(s="Cohort Month:", x=legend_x, y=legend_y + (.05 * i) + .05, weight='bold')
plt.title("365-Day Daily Retention")
plt.xlabel("Days Since User First Started")
plt.ylabel("Percent Of Users Retained")
plt.ylim(0,1.05)
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]);

In [ ]:
data.loc[ data['visitcount'] != 1, ['visit_type']] = 'Old'
data.loc[ data['visitcount'] == 1, ['visit_type']] = 'New'

In [ ]:
data['visit_type'].value_counts()


In [ ]:
Months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
             'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


sns.barplot(data = data, x='month',y='Invoice_Value',estimator = 'sum' ,errorbar=None,order=Months,hue='visit_type')
plt.ylabel('Revenue')

In [ ]:
discountrole =  pd.DataFrame(data.groupby(by ='month')['Coupon_Status'].value_counts())
discountrole=discountrole.reset_index()

In [ ]:
from pandas.api.types import CategoricalDtype

cat_month = CategoricalDtype(
    ['Jan', 'Feb', 'Mar', 'Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
    ordered=True,
)

In [ ]:
discountrole['month']=discountrole['month'].astype(cat_month)
data['month']=data['month'].astype(cat_month)

In [ ]:
sns.lineplot(data=discountrole,x='month',y='count', hue ='Coupon_Status')
plt.ylabel("Orders")

#From the below it can be clearlt seen that the sale is higher when dicount is present

In [ ]:
avg_revenue = pd.pivot_table(data=data,index=['month','visit_type'],values =['Invoice_Value','Transaction_ID'],aggfunc={'Invoice_Value':'sum','Transaction_ID':'count'})

In [ ]:
avg_revenue=avg_revenue.reset_index()

In [ ]:
avg_revenue['avg revenue per order']=avg_revenue['Invoice_Value']/avg_revenue['Transaction_ID']

In [ ]:
data.groupby('Product_Category')['Invoice_Value'].sum().plot()

In [ ]:
data.groupby('Location')['Invoice_Value'].sum().plot()

In [ ]:
weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
sns.barplot(data = data, x=data['Transaction_Date'].dt.day_name(),y='Invoice_Value',order=weekdays)
plt.ylabel("Avg Revenue")
plt.xlabel("Weekday")

In [ ]:
ms['month']=ms['Date'].dt.strftime('%m')

In [ ]:
#ms.groupby(ms['Date'].dt.strftime("%m"))[['Offline_Spend','Online_Spend']].sum()

In [ ]:
marketspend = pd.DataFrame(ms.groupby('month')[['Offline_Spend','Online_Spend']].sum())

In [ ]:
marketspend['totalspend']=marketspend['Offline_Spend']+marketspend['Online_Spend']

In [ ]:
marketspend.reset_index(inplace=True)

In [ ]:
market = pd.DataFrame(data.groupby('month')[['Invoice_Value','GST','Delivery_Charges']].sum())
market.reset_index(inplace=True)

In [ ]:
marketspend.drop(columns='month',inplace=True)

In [ ]:
monthly=pd.concat([market,marketspend],axis='columns')

In [ ]:
monthly['%marketingspend']=round(monthly['totalspend']*100/monthly['Invoice_Value'],0)

In [ ]:
monthly['%delivrycharge']=round(monthly['GST']*100/monthly['Invoice_Value'],1)

In [ ]:
monthly

In [ ]:
monthly[['Invoice_Value','totalspend']].corr()
#From the below we can see that there is a strong correlation between market spend and revenue generated

In [ ]:
#Product Appeared in Transactions
data['Product_Description'].unique()

In [ ]:
data['Product_Description'].value_counts()[:1].index.tolist()

In [ ]:
CustomerRFM = pd.pivot_table(data,index=['CustomerID'],values=['Transaction_Date','Transaction_ID','Invoice_Value'],aggfunc={'Transaction_Date':'max','Transaction_ID':'count','Invoice_Value':'sum'})

In [ ]:
last_date = '2020-01-01'
last_date=pd.to_datetime(last_date)

In [ ]:
CustomerRFM['Recency'] = CustomerRFM['Transaction_Date'].apply(lambda x: last_date-x).dt.days

In [ ]:
CustomerRFM.rename(columns={'Transaction_ID':'visitcount','Invoice_Value':'Monetary'},inplace=True)

In [ ]:
CustomerRFM.drop(columns='Transaction_Date',inplace=True)

In [ ]:
sns.histplot(CustomerRFM['Monetary'])

In [ ]:
sns.histplot(CustomerRFM['visitcount'])

In [ ]:
sns.histplot(CustomerRFM['Recency'])

In [ ]:
print(CustomerRFM['Recency'].quantile(.25))
print(CustomerRFM['Recency'].quantile(.50))

In [ ]:
def R_Score(x):
    x['Recency'].astype('int')
    if x['Recency']<56:
        recency = 3
    elif x['Recency']>=132:
        recency = 1
    else:
        recency = 2
    return recency

In [ ]:
CustomerRFM['R'] = CustomerRFM.apply(R_Score,axis=1)

In [ ]:
#R_Score = 3-pd.qcut(CustomerRFM['Recency'],q=3,labels= False)
#CustomerRFM = CustomerRFM.assign(R = R_Score.values)
#F_Score = 1+pd.qcut(CustomerRFM['visitcount'],q=3,labels= range(1,4))

F_Score = 1+pd.qcut(CustomerRFM['visitcount'],q=3,labels= False)
CustomerRFM = CustomerRFM.assign(F = F_Score.values)

M_Score = 1+pd.qcut(CustomerRFM['Monetary'],q=3,labels= False)
CustomerRFM = CustomerRFM.assign(M = M_Score.values)

In [ ]:
CustomerRFM.dtypes

In [ ]:
CustomerRFM['R'].astype('str')

In [ ]:
def RFM_Score(x):
    return str(x['R']) + str(x['F']) + str(x['M'])
CustomerRFM['RFM_Score'] = CustomerRFM.apply(RFM_Score,axis=1)

In [ ]:
CustomerRFM['RFM_Score'] = CustomerRFM.apply(RFM_Score,axis=1)

In [ ]:
CustomerRFM['RFM_Score']=CustomerRFM['RFM_Score'].astype('int')

In [ ]:
CustomerRFM['RFM_Score'].sort_values().unique()

In [ ]:
def RFM_Segment(x):
    if x['RFM_Score']==333:
        Segment = 'Premium'
    elif x['RFM_Score'] in (133,233,313,323,331,332):
        Segment = 'Gold'
    elif x['RFM_Score'] in (113,123,213,223,131,132,231,231,311,312,321,322):
        Segment = 'Silver'
    else:
        Segment = 'Standard'
    return Segment

In [ ]:
CustomerRFM['RFM_Segment']=CustomerRFM.apply(RFM_Segment,axis=1)

In [ ]:
CustomerRFM['RFM_Segment'].value_counts()

In [ ]:
CustomerRFM2=CustomerRFM[['Monetary','visitcount','Recency']]

In [ ]:
#Using  K-means
from sklearn.cluster import KMeans

distortions = []
K = range(1,7)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(CustomerRFM2)
    distortions.append(kmeanModel.inertia_)

In [ ]:
#plot elbow graph
plt.figure(figsize=(15,5))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method Showing The Optimal k')
sns.despine()
plt.show()

In [ ]:
#We choose k value to be 3
clusters = KMeans(n_clusters=3, random_state=42).fit(CustomerRFM2)
CustomerRFM2["Kman Segmentation"] = clusters.labels_

In [ ]:
CustomerRFM2["Kman Segmentation"].value_counts()

In [ ]:
g = sns.FacetGrid(CustomerRFM2,
                  row="Kman Segmentation",
                  hue="Kman Segmentation",
                  aspect=10,
                  # palette=palette
                  )
# Draw the densities in a few steps
g.map(sns.kdeplot, "Monetary",
      bw_adjust=.5, clip_on=False,
      fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, "Monetary", clip_on=False, color="w", lw=2, bw_adjust=.5, alpha=.7)

# passing color=None to refline() uses the hue mapping
g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(Monetary, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "Monetary")

# Add vertical lines for mean age on each plot
def vertical_mean_line_survived(x, **kwargs):
    plt.axvline(x.mean(), linestyle ="-", 
                color = "black", alpha=.5)
    txkw = dict(size=10, color = "black")
    tx = "mean: {:.2f} | std: {:.2f}".format(x.mean(),x.std())
    ax = plt.gca()
    ax.text(0.06+x.mean()/175000, .2, tx, color="black", size= 10,
            ha="left", va="center", transform=ax.transAxes)
    # plt.text(x.mean()+1, 0.0052, tx, **txkw)

g.map(vertical_mean_line_survived, 'Monetary')

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=0.1)

# Remove axes details that don't play well with overlap
g.fig.suptitle('Income Distributions For Customer Personalities')
g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

In [ ]:
g = sns.FacetGrid(CustomerRFM2,
                  row="Kman Segmentation",
                  hue="Kman Segmentation",
                  aspect=10,
                  # palette=palette
                  )
# Draw the densities in a few steps
g.map(sns.kdeplot, "visitcount",
      bw_adjust=.5, clip_on=False,
      fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, "visitcount", clip_on=False, color="w", lw=2, bw_adjust=.5, alpha=.7)

# passing color=None to refline() uses the hue mapping
g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(visitcount, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "visitcount")

# Add vertical lines for mean age on each plot
def vertical_mean_line_survived(x, **kwargs):
    plt.axvline(x.mean(), linestyle ="-", 
                color = "black", alpha=.5)
    txkw = dict(size=10, color = "black")
    tx = "mean: {:.2f} | std: {:.2f}".format(x.mean(),x.std())
    ax = plt.gca()
    ax.text(0.06+x.mean()/175000, .2, tx, color="black", size= 10,
            ha="left", va="center", transform=ax.transAxes)
    # plt.text(x.mean()+1, 0.0052, tx, **txkw)

g.map(vertical_mean_line_survived, 'visitcount')

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=0.1)

# Remove axes details that don't play well with overlap
g.fig.suptitle('Income Distributions For Customer Personalities')
g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

In [ ]:
g = sns.FacetGrid(CustomerRFM2,
                  row="Kman Segmentation",
                  hue="Kman Segmentation",
                  aspect=10,
                  # palette=palette
                  )
# Draw the densities in a few steps
g.map(sns.kdeplot, "Recency",
      bw_adjust=.5, clip_on=False,
      fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, "Recency", clip_on=False, color="w", lw=2, bw_adjust=.5, alpha=.7)

# passing color=None to refline() uses the hue mapping
g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(Recency, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "Recency")

# Add vertical lines for mean age on each plot
def vertical_mean_line_survived(x, **kwargs):
    plt.axvline(x.mean(), linestyle ="-", 
                color = "black", alpha=.5)
    txkw = dict(size=10, color = "black")
    tx = "mean: {:.2f} | std: {:.2f}".format(x.mean(),x.std())
    ax = plt.gca()
    ax.text(0.06+x.mean()/175000, .2, tx, color="black", size= 10,
            ha="left", va="center", transform=ax.transAxes)
    # plt.text(x.mean()+1, 0.0052, tx, **txkw)

g.map(vertical_mean_line_survived, 'Recency')

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=0.1)

# Remove axes details that don't play well with overlap
g.fig.suptitle('Income Distributions For Customer Personalities')
g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

# from the above we can conclude that segment 3 have low recency high frequency and high monetary value and segment 1 has the lowest of all values while segment 2 has mid of the two segment

#Customer Lifetime Value = Avg. Customer Lifespan* Customer Value

Customer Value = Average Purchase Value X Average No of Purchases

Average Purchase Value = Total Revenue/no of orders
Average No of Purchases = No of purchases/No of customer

Avg. Customer Lifespan = Sum of customer lifespan / no of customer

In [ ]:
lftv = pd.pivot_table(data=data,index=['CustomerID','Location','Gender'],values=['month','Transaction_Date','Tenure_Months','Invoice_Value'],aggfunc={'month':'min','Tenure_Months':'median','Invoice_Value':'sum'})

In [ ]:
lftv.reset_index(inplace=True)

In [ ]:
lftv.head()

In [ ]:
lftv['basket']=pd.qcut(x=lftv['Invoice_Value'], q=3,labels=['Low Value','Medium Value','High Value'])

In [ ]:
lftv.drop(columns = ['Invoice_Value','CustomerID'],inplace=True)

In [ ]:
lftv_model =pd.get_dummies(lftv,prefix={'Location':'location', 'Gender':'Gender', 'month':'month'},columns = ['Location','Gender','month'],drop_first=False)

In [ ]:
#permorning classification model
x=lftv_model.copy()
y=x.pop('basket')

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
x=scale.fit_transform(x)

from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(x,y)
ypred = LR.predict(x)

#Takes real long time to run!! hence markdowned

#Finding Cross selling product
grouped = pd.crosstab(data['Transaction_ID'],data['Product_SKU'])
grouped=grouped.reset_index()
grouped.drop(columns='Transaction_ID',inplace=True)

mygroup = pd.DataFrame(columns=['col1','freq'])

from collections import Counter
from itertools import combinations

for i in combinations(grouped.columns,2):
    x,y = i
    mygroup.loc[len(mygroup),['col1','freq']]=(x,y),len(grouped[(grouped[x]==grouped[y])&(grouped[x]==1)])

for i in combinations(grouped.columns,3):
    x,y,z = i
    mygroup.loc[len(mygroup),['col1','freq']]=(x,y),len(grouped[(grouped[x]==grouped[y])&(grouped[x]==grouped[z])&(grouped[x]!=0)])

mygroup.head(10).sort_values(by='freq',ascending=False)

In [ ]:
def max_min(x):
    mx=x.max()
    mn=x.min()
    y= (mx-mn).days
    return y

In [ ]:
avgtenure=pd.DataFrame(pd.pivot_table(data=data,index=['CustomerID'],values=['Transaction_Date'],aggfunc={'Transaction_Date':[pd.Series.nunique,'max','min',max_min]}))

In [ ]:
avgtenure.reset_index(inplace=True)

In [ ]:
avgtenure.columns=avgtenure.columns.droplevel()

In [ ]:
avgtenure.head()

In [ ]:
avgtenure.reset_index(inplace=True)
avgtenure.rename(columns={'max_min':'nofdays','nunique':'N of Trans'},inplace=True)

In [ ]:
avgtenure['avglife']=round(avgtenure['nofdays']/avgtenure['N of Trans'],0)

In [ ]:
sns.displot(data=avgtenure['avglife'])

In [ ]:
data=pd.merge(left=data,right=avgtenure,on='CustomerID',how='left')

In [ ]:
data['NPD']=pd.cut(x=data['avglife'], bins=[0,30,60,90,200],labels=['0-30','30-60','60-90','>90'])

In [ ]:
sns.heatmap(data.select_dtypes('int','float').corr())

In [ ]:
data2=data.copy()

In [ ]:
data2=data2[(data2['avglife']>0)&(data2['visitcount']>1)]

In [ ]:
data2.reset_index(inplace=True)
data2.drop(columns={'index','CustomerID','Transaction_ID'},inplace=True)

In [ ]:
data2['NPD']=data2['NPD'].astype('str')

In [ ]:
#Selecting feature for target variable
x2train=data2[['Product_Description','Quantity','Coupon_Status','Location','Tenure_Months','visitcount','Gender','NPD']]


In [ ]:
y2train=x2train.pop('NPD')

In [ ]:
x2train =pd.get_dummies(x2train,prefix={'Product_Description':'Product_Description', 'Coupon_Status':'Coupon_Status','Location':'Location','Gender':'Gender'},columns = ['Product_Description','Coupon_Status','Location','Gender'],drop_first=False)

In [ ]:
x2train.dtypes

In [ ]:
x2train.isnull().sum()

In [ ]:
y2train.isnull().sum()

In [ ]:
y2train.dtypes

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
x2train=scale.fit_transform(x2train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
LR = RandomForestClassifier()
LR.fit(x2train,y2train)
ypred = LR.predict(x2train)

In [ ]:
data.columns

In [ ]:
data['start_month']=data['start_day'].dt.strftime('%b')

In [ ]:
#New customer for each month cohort
data[data['visitcount']==1].groupby('start_month')['CustomerID'].nunique()

In [ ]:
#Total transaction for each month cohort
data.groupby('start_month')['Transaction_ID'].nunique()

In [ ]:
#Total Value purchased by monthly cohort
data.groupby('start_month')['Invoice_Value'].sum()

In [ ]:
#which month has maximum retention
data[data['months_since_start']==1].groupby('start_month')['CustomerID'].nunique()

In [ ]:
data[data['months_since_start']==1].groupby('start_month')['CustomerID'].nunique().idxmax()